# Dataframe validaciones mensuales

In [139]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import unicodedata

ds = 'troncal_2024.csv'
df = pd.read_csv(ds)
df2= df.copy()
df2.head()

,Estacion_Parada,latitud,longitud,fecha,hora,Sistema,validaciones
0,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-10,19,DUAL,1
1,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-11,19,DUAL,7
2,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-12,17,DUAL,9
3,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-12,19,DUAL,5
4,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-12,20,DUAL,1


In [140]:
df.shape

(1628608, 7)

In [141]:
df.describe(include='all')

,Estacion_Parada,latitud,longitud,fecha,hora,Sistema,validaciones
count,1628608,1.621489e+06,1.621489e+06,1628608,1.628608e+06,1628608,1.628608e+06
unique,718,NaN,NaN,366,NaN,2,NaN
top,(08000) Portal Tunal,NaN,NaN,2024-01-26,NaN,TRONCAL,NaN
freq,7923,NaN,NaN,4808,NaN,1090620,NaN
mean,NaN,4.647300e+00,-7.408770e+01,NaN,1.327862e+01,NaN,3.602492e+02
std,NaN,5.174985e-02,3.630330e-02,NaN,5.539746e+00,NaN,8.261560e+02
min,NaN,4.528831e+00,-7.420483e+01,NaN,0.000000e+00,NaN,1.000000e+00
25%,NaN,4.604883e+00,-7.411167e+01,NaN,9.000000e+00,NaN,1.700000e+01
50%,NaN,4.646547e+00,-7.408050e+01,NaN,1.300000e+01,NaN,1.010000e+02
75%,NaN,4.689507e+00,-7.406125e+01,NaN,1.800000e+01,NaN,3.580000e+02


In [142]:
df2["latitud"] = pd.to_numeric(df2["latitud"], errors="coerce")
faltan_lat = df2[df2["latitud"].isna()]
faltan_lat.head()

,Estacion_Parada,latitud,longitud,fecha,hora,Sistema,validaciones
1614167,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,10,TRONCAL,376
1614168,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,11,TRONCAL,533
1614169,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,12,TRONCAL,561
1614170,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,13,TRONCAL,480
1614171,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,14,TRONCAL,504


In [143]:
#Para saber si todos los datos fantantes son de la misma estación
solo_una = faltan_lat['Estacion_Parada'].nunique() == 1
print(solo_una)   

True


In [144]:
estacion = "Corral Portal Dorado"     
nueva_lat = 4.680867
nueva_lon = -74.120859

# Para encontrar la parada en el df
est = df2['Estacion_Parada'].str.contains(estacion)

# Para rellenar donde actualmente hay NaN 
df2.loc[est & df2['latitud'].isna(), 'latitud'] = nueva_lat
df2.loc[est & df2['longitud'].isna(), 'longitud'] = nueva_lon

In [145]:
FECHA_COL = "fecha"
LAT_COL = "latitud"
LON_COL = "longitud"
VAL_COL = "validaciones"
EST_COL = "Estacion_Parada"  

LOC_NAME_COL = "LocNombre"

# Archivo de salida
OUT_CSV = "validaciones_localidad_mensual_2024.csv"

base = df2.copy()

# Tipos
base[FECHA_COL] = pd.to_datetime(base[FECHA_COL], errors="coerce")
base[LAT_COL]   = pd.to_numeric(base[LAT_COL], errors="coerce")
base[LON_COL]   = pd.to_numeric(base[LON_COL], errors="coerce")
base[VAL_COL]   = pd.to_numeric(base[VAL_COL], errors="coerce")

# Filtra nulos duros
base = base.dropna(subset=[FECHA_COL, LAT_COL, LON_COL, VAL_COL]).copy()

# Partes de fecha
base["año"] = base[FECHA_COL].dt.year
base["mes"] = base[FECHA_COL].dt.month

# (Opcional) Recorte geográfico
base = base[base[LAT_COL].between(LAT_MIN, LAT_MAX) & base[LON_COL].between(LON_MIN, LON_MAX)].copy()

base.head()

,Estacion_Parada,latitud,longitud,fecha,hora,Sistema,validaciones,año,mes
0,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-10,19,DUAL,1,2024,1
1,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-11,19,DUAL,7,2024,1
2,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-12,17,DUAL,9,2024,1
3,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-12,19,DUAL,5,2024,1
4,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-12,20,DUAL,1,2024,1


In [ ]:
SHP_FILE = r"C:\Users\PaolaCastro\Maestría\Proyecto\Localidades\Loca.shp"

# Archivo de salida
OUT_CSV = "validaciones_localidad_mensual_2024.csv"
base = df2.copy()

# Tipos
base[FECHA_COL] = pd.to_datetime(base[FECHA_COL], errors="coerce")
base[LAT_COL] = pd.to_numeric(base[LAT_COL], errors="coerce")
base[LON_COL] = pd.to_numeric(base[LON_COL], errors="coerce")
base[VAL_COL] = pd.to_numeric(base[VAL_COL], errors="coerce")

# Partes de fecha
base["año"] = base[FECHA_COL].dt.year
base["mes"] = base[FECHA_COL].dt.month

# Para cargar localidades y CRS
gdf_loc = gpd.read_file(SHP_FILE)

# Como cartografía de Bogotá viene en EPSG:4686
if gdf_loc.crs is None:
    gdf_loc = gdf_loc.set_crs(4686, allow_override=True)
gdf_loc = gdf_loc.to_crs(4326)

# Puntos desde 'base' y a sistema métrico (EPSG:3116)
gdf = gpd.GeoDataFrame(base, geometry=gpd.points_from_xy(base[LON_COL], base[LAT_COL]), crs=4326)
gdf_m = gdf.to_crs(3116)
gdf_loc_m = gdf_loc.to_crs(3116)

# Para asignar los puntos que están claramente dentro de una localidad
est_dentro = gpd.sjoin(gdf_m, gdf_loc_m[[LOC_NAME_COL, "geometry"]], how="left", predicate="within")

con_loc = est_dentro[~est_dentro[LOC_NAME_COL].isna()].copy()             # puntos con localidad por 'within'
sin_loc = est_dentro[est_dentro[LOC_NAME_COL].isna()].drop(columns=[LOC_NAME_COL])  # sin localidad (a rescatar)

# Para los que quedan sin una, se le asigna a la localidad más cercana
tol_m = 25  # metros
near = gpd.sjoin_nearest(sin_loc, gdf_loc_m[[LOC_NAME_COL, "geometry"]], how="left", max_distance=tol_m, distance_col="dist", lsuffix="L", rsuffix="R")

# Si un punto está con más de una, se queda con la más cercana
near = (near.reset_index().sort_values(['index', 'dist']).drop_duplicates(subset='index', keep='first').set_index('index'))

# Combina los que están claramente y los quye se asignaron por cercanía, y vuelve a 4326
combined_m = pd.concat([matched_m, near], axis=0).sort_index()
combined = (combined_m.to_crs(4326).rename(columns={LOC_NAME_COL: "localidad"}).drop(columns=["dist", "index_right", "index_left"], errors="ignore"))

#Como Soacha no está en el SHP asigno las estaciones que están en esa localidad
soacha = {"(07503) SAN MATEO - C.C. UNISUR": "SOACHA", "(07504) TERREROS": "SOACHA", "(07505) LEON XIII": "SOACHA", "(07506) DESPENSA": "SOACHA"}
combined["localidad"] = combined["localidad"].mask(combined[EST_COL].isin(soacha),combined[EST_COL].map(soacha))

# Agregación mensual por localidad 
valid_loc_mes = (combined.groupby(["año","mes","localidad"], as_index=False)[VAL_COL].sum().rename(columns={VAL_COL: "validaciones_mensuales"}).sort_values(["año","mes","localidad"]))

# Exporta
valid_loc_mes.to_csv(OUT_CSV, index=False, encoding="utf-8")
print(f"Exportado: {OUT_CSV}")
valid_loc_mes.head()


In [137]:
#Ahora quiero validad que no se perdieron registros. Es decir, que las validaciones que me dieron por localidad son laas mismas que el total de validaciones
total1= valid_loc_mes["validaciones_mensuales"].sum()
print(total1)
total = df2["validaciones"].sum()
print(total)

586704658
586704658
